> ### Note on Labs and Assignments:
>
> 🔧 Look for the **wrench emoji** 🔧 — it highlights where you're expected to take action!
>
> These sections are graded and are not optional.
>

# IS 4487 Lab 7: Data Transformation

## Outline

- Load and preview the cleaned Megatelco dataset  
- Engineer new columns from existing data  
- Simplify or group variable values  
- Use `.map()`, `.apply()`, and `pd.cut()` for transformations  
- Try your own transformation logic  

This lab continues from **Lab 6**, where we cleaned the Megatelco dataset.  

Now, we will create new, more useful features for modeling and exploration.

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/lab_07_data_transformation.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## Megatelco Data Dictionary

 DEMOGRAPHIC VARIABLES:
 - College - has the customer attended some college (one, zero)
 - Income - annual income of customer
 - House - estimated price of the customer's home (if applicable)

 USAGE VARIABLES:
 - Data Overage Mb - Average number of megabytes that the customer used in excess of the plan limit (over last 12 months)
 - Data Leftover Mb - Average number of megabytes that the customer use was below the plan limit (over last 12 months)
 - Data Mb Used - Average number of megabytes used per month (over last 12 months)
 - Text Message Count - Average number of texts per month (over last 12 months)
 - Over 15 Minute Calls Per Month - Average number of calls over 15 minutes in duration per month (over last 12 months)
 - Average Call Duration- Average call duration (over last 12 months)

PHONE VARIABLES:
 - Operating System - Current operating system of phone
 - Handset Price - Retail price of the phone used by the customer

ATTITUDINAL VARIABLES:
 - Reported Satisfaction - Survey response to "How satisfied are you with your current phone plan?" (high, med, low)
 - Reported Usage Level - Survey response to "How much do your use your phone?" (high, med, low)
 - Considering Change of Plan - Survey response to "Are you currently planning to change companies when your contract expires?" (high, med, low)

OTHER VARIABLES
 - Leave - Did this customer churn with the last contract expiration? (LEAVE, STAY)
 - ID - Customer identifier

# Part 1: Data Cleaning steps from Lab 6

In this part of the lab, we will load the cleaning steps previously done in lab 6

- Load the Megatelco dataset
- Clean column names
- Fix data types
- Handle missing values
- Remove duplicate records
- Review for outliers


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = "https://raw.githubusercontent.com/Stan-Pugsley/is_4487_base/812e9f15c357a5657a2795631fcaa9d9363cb417/DataSets/megatelco_leave_survey_data_cleaning_v2.csv"
df = pd.read_csv(url)

df.head()

,college,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,reported_satisfaction,reported_usage_level,considering_change_of_plan,leave,id,operating_system
0,one,403137.0,70,0.0,6605.0,199,841317,653.0,5.0,8.0,low,low,yes,LEAVE,8183,Android
1,zero,129700.0,67,16.0,6028.0,134,476664,1193.0,5.0,5.0,low,low,yes,LEAVE,12501,IOS
2,zero,69741.0,60,0.0,1482.0,176,810225,1037.0,3.0,8.0,low,low,yes,STAY,7425,IOS
3,one,377572.0,0,22.0,3005.0,184,826967,1161.0,0.0,5.0,low,low,no,LEAVE,13488,IOS
4,zero,382080.0,0,0.0,1794.0,74,951896,1023.0,0.0,14.0,low,low,yes,STAY,11389,IOS


In [2]:
#check datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   college                      15016 non-null  object 
 1   income                       15006 non-null  float64
 2   data_overage_mb              15016 non-null  int64  
 3   data_leftover_mb             14916 non-null  float64
 4   data_mb_used                 14916 non-null  float64
 5   text_message_count           15016 non-null  int64  
 6   house                        15016 non-null  int64  
 7   handset_price                14916 non-null  float64
 8   over_15mins_calls_per_month  15013 non-null  float64
 9   average_call_duration        14916 non-null  float64
 10  reported_satisfaction        15016 non-null  object 
 11  reported_usage_level         15016 non-null  object 
 12  considering_change_of_plan   14201 non-null  object 
 13  leave           

### Standardize column names
 - Remove leading/trailing whitespace
 - Convert to lowercase
 - Replace spaces with underscores

In [3]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Show info about data types and non-null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   college                      15016 non-null  object 
 1   income                       15006 non-null  float64
 2   data_overage_mb              15016 non-null  int64  
 3   data_leftover_mb             14916 non-null  float64
 4   data_mb_used                 14916 non-null  float64
 5   text_message_count           15016 non-null  int64  
 6   house                        15016 non-null  int64  
 7   handset_price                14916 non-null  float64
 8   over_15mins_calls_per_month  15013 non-null  float64
 9   average_call_duration        14916 non-null  float64
 10  reported_satisfaction        15016 non-null  object 
 11  reported_usage_level         15016 non-null  object 
 12  considering_change_of_plan   14201 non-null  object 
 13  leave           

### Convert Datatypes

These steps were used in Lab 6.  We will use them again to get a usable dataframe.

In [4]:
# Check original data types
print("Original dtypes:\n", df.dtypes)

# Convert yes/no columns to binary categories
df['considering_change_of_plan'] = df['considering_change_of_plan'].map({'yes': 1, 'no': 0}).astype('category')
df['leave'] = df['leave'].map({'yes': 1, 'no': 0}).astype('category')  # newly added

# Convert categorical text columns to 'category' dtype
df['college'] = df['college'].astype('category')
df['reported_satisfaction'] = df['reported_satisfaction'].astype('category')
df['reported_usage_level'] = df['reported_usage_level'].astype('category')  # newly added
df['operating_system'] = df['operating_system'].astype('category')

# Convert object/text columns with limited possible values with an order to ordinal categorical columns
df['reported_satisfaction'] = pd.Categorical(df['reported_satisfaction'], categories = ['Low', 'Medium', 'High'], ordered = True)
df['reported_usage_level'] = pd.Categorical(df['reported_usage_level'], categories = ['Low', 'Medium', 'High'], ordered = True)

# Convert binary columns ('yes'/'no', 'LEAVE'/'STAY') to binary categorical
df['considering_change_of_plan'] = df['considering_change_of_plan'].astype('category')
df['leave'] = df['leave'].astype('category')

# Check updated data types
print("\nUpdated dtypes:\n", df.dtypes)


Original dtypes:
 college                         object
income                         float64
data_overage_mb                  int64
data_leftover_mb               float64
data_mb_used                   float64
text_message_count               int64
house                            int64
handset_price                  float64
over_15mins_calls_per_month    float64
average_call_duration          float64
reported_satisfaction           object
reported_usage_level            object
considering_change_of_plan      object
leave                           object
id                               int64
operating_system                object
dtype: object

Updated dtypes:
 college                        category
income                          float64
data_overage_mb                   int64
data_leftover_mb                float64
data_mb_used                    float64
text_message_count                int64
house                             int64
handset_price                   float64
over_1

## Part 2: Creating New Features

A major part of data preparation is **feature engineering** — creating new columns from raw data to capture useful patterns.

In this section, we will try three common methods:

1. `.map()` — useful for simplifying categories (e.g., satisfaction levels)
2. `.apply()` — allows flexible custom logic (e.g., flagging high usage)
3. `pd.cut()` or `pd.qcut()` — groups numeric values into bins or quantiles

These new features help models learn better and make reports easier to interpret.

Things to think about:
- Are any categories too specific or inconsistent?
- Can you create groups or flags to highlight important traits?
- Would a simplified version of a column help with modeling or visualization?

In [5]:
# Create a total data usage variable (used + leftover)
df['total_data_mb'] = df['data_mb_used'] + df['data_leftover_mb']

# Create a ratio of overage to used data
df['overage_ratio'] = df['data_overage_mb'] / (df['data_mb_used'] + 1)  # add 1 to avoid divide-by-zero

# Create a binary flag for high texters (over 500 texts)
df['high_texter'] = (df['text_message_count'] > 500).astype(int)

# Preview new columns
df[['total_data_mb', 'overage_ratio', 'high_texter']].head()


,total_data_mb,overage_ratio,high_texter
0,6605.0,0.010596,0
1,6044.0,0.011113,0
2,1482.0,0.040459,0
3,3027.0,0.000000,0
4,1794.0,0.000000,0


### 🔧 Try It Yourself – Part 2

1. Create a variable called `call_volume` by multiplying `over_15mins_calls_per_month` by `average_call_duration`
2. Create a binary flag `high_data_user` for users where `data_mb_used` is above the median
3. Use `.head()` to check your new columns



In [8]:
# 🔧 Add code here
df['call_volume'] = df['over_15mins_calls_per_month'] * df['average_call_duration']
df['high_data_user'] = (df['data_mb_used'] > df['data_mb_used'].median()).astype(int)

# Preview new columns
df[['call_volume', 'high_data_user']].head()

,call_volume,high_data_user
0,40.0,1
1,25.0,1
2,24.0,0
3,0.0,0
4,0.0,0


## Part 3: Binning Continuous Variables

Binning is the process of grouping numeric variables into categories (e.g., "low", "medium", "high").

### Why We Bin:
- Helps reduce the impact of outliers
- Allows us to use numeric values in models that prefer categories
- Simplifies interpretation and visualization

### Things to think about:
- Would grouping values make patterns more visible?
- Do we want equal-sized groups or logical cutoffs?
- Is the variable skewed?

**Tools:**  
- `pd.qcut()` for quantile-based bins (equal frequency)  
- `pd.cut()` for equal-width or custom bins


In [9]:
# Bin income into 3 groups (quantiles): Low, Medium, High
df['income_group'] = pd.qcut(df['income'], q=3, labels=['Low', 'Medium', 'High'])

# Bin average call duration into quartiles (labels as integers)
df['call_duration_group'] = pd.qcut(df['average_call_duration'], q=4, labels=False)

# Preview new groupings
df[['income', 'income_group', 'average_call_duration', 'call_duration_group']].head()

,income,income_group,average_call_duration,call_duration_group
0,403137.0,High,8.0,1.0
1,129700.0,Low,5.0,0.0
2,69741.0,Low,8.0,1.0
3,377572.0,High,5.0,0.0
4,382080.0,High,14.0,2.0


### 🔧 Try It Yourself – Part 3

1. Use `pd.cut()` to group `data_mb_used` into 3 labeled bins: "Light", "Moderate", "Heavy"
2. Use `pd.qcut()` on `text_message_count` to split into 4 equal-sized groups
3. Print `.value_counts()` on each new column to see how values are distributed

In [10]:
# 🔧 Add code here
df['data_mb_used_group'] = pd.cut(df['data_mb_used'], bins=3, labels=['Light', 'Moderate', 'Heavy'])
df['text_message_count_group'] = pd.qcut(df['text_message_count'], q=4, labels=False)

print(df['data_mb_used_group'].value_counts())

data_mb_used_group
Heavy       5031
Light       4993
Moderate    4892
Name: count, dtype: int64


## Part 4: Scaling Numeric Variables

Scaling transforms values to a common range (often 0–1), which helps many machine learning models perform better.

### When to Scale:
- When features have very different ranges (e.g., income vs. call duration)
- When using distance-based models (e.g., KNN, SVM)
- When comparing magnitudes across features

### Common Methods:
- `MinMaxScaler`: scales to 0–1 range
- `StandardScaler`: centers data around 0 with unit variance

### Things to think about:
- Are features on different scales?
- Does my algorithm care about magnitude?

In [11]:
from sklearn.preprocessing import MinMaxScaler

# Choose columns to scale
cols_to_scale = ['income', 'data_mb_used', 'average_call_duration']

# Initialize and apply scaler
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[cols_to_scale])

# Add scaled columns back to df
df['income_scaled'] = df_scaled[:, 0]
df['data_mb_used_scaled'] = df_scaled[:, 1]
df['avg_call_dur_scaled'] = df_scaled[:, 2]

# Preview
df[['income_scaled', 'data_mb_used_scaled', 'avg_call_dur_scaled']].head()

,income_scaled,data_mb_used_scaled,avg_call_dur_scaled
0,0.941926,0.816447,0.001400
1,0.391751,0.740526,0.000800
2,0.271109,0.142368,0.001400
3,0.890487,0.342763,0.000800
4,0.899557,0.183421,0.002601


### 🔧 Try It Yourself – – Part 4

1. Scale the `handset_price` and `over_15mins_calls_per_month` columns using `MinMaxScaler`
2. Add the scaled values back to the dataframe using 2 new columns with suffix `_scaled`
3. Use `.describe()` to compare original vs. scaled versions  

### In Your Response:
1. Make a comment on what you observe your comparison


In [12]:
# 🔧 Add code here
from sklearn.preprocessing import MinMaxScaler
cols_to_scale = ['handset_price', 'over_15mins_calls_per_month']

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[cols_to_scale])

df['handset_price'] = df_scaled[:, 0]
df['over_15mins_calls_per_month'] = df_scaled[:, 1]

df[['handset_price', 'over_15mins_calls_per_month']].describe()


,handset_price,over_15mins_calls_per_month
count,14916.000000,15013.000000
mean,0.007947,0.301872
std,0.009896,0.240120
min,0.000000,0.000000
25%,0.005575,0.085714
50%,0.007804,0.257143
75%,0.010088,0.485714
max,1.000000,1.000000


### ✍️ Your Response: 🔧
1. The range becomes smaller and makes the data easier to manage. The max and mins are the same for both variables.

## Part 5: Encoding Categorical Variables

Most machine learning models can't handle string categories directly—so we convert them into numbers using **encoding**.

### Types of Encoding:
- **One-hot encoding**: creates a binary column for each unique value in the original categorical column (**used for nominal variables**)
  - an example: We have a column called Colors with 3 values: red, blue, green - that we want to treat as nominal. OHE will create 3 new columns corresponidng to each color, and place a 1 in the new column, wherever the value in original column matches the color in its colname.
  - The original column is dropped by default, so it adds n new colums where n = number of unique values

|Color   |   red  | blue  | green  |
|--------|--------|-------|--------|
|red     |    1   |   0   |    0   |
|blue    |    0   |   1   |    0   |
|green   |    0   |   0   |    1   |
|red     |    1   |   0   |    0   |

- **Ordinal encoding**: assigns integers (**use only for ordered categories**)
  - An example: if we have a column called Education, we may want to set their values as 1,2,3,.....
  - adds only one new column, original column remains.

|Education   | educ_int  |
|--------|--------|
|some high school |     1     |
|high school grad |     2    |
|some college  |     3     |
|college grad |     4    |

### Things to consider:
- Is the variable nominal (e.g., OS type) or ordinal (e.g., satisfaction)?
- How many unique categories are there?
- Will one-hot encoding make the dataset too wide?

**Tool:** `pd.get_dummies()`

In [13]:
# One-hot encode 'reported_usage_level'
df_encoded = pd.get_dummies(df, columns=['reported_usage_level'], prefix='usage')

# One-hot encode 'income_group'
df_encoded = pd.get_dummies(df_encoded, columns=['income_group'], prefix='income')

# Preview new columns
df_encoded.filter(like='usage_').head()

,usage_Low,usage_Medium,usage_High
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False


### 🔧 Try It Yourself – Part 5

1. One-hot encode `reported_satisfaction` and `operating_system`
2. Print `.shape` of your dataframe before and after to observe any big changes

### In Your Response:
1. How many new columns were added?


In [19]:
# 🔧 Add code here
# Store the shape before this cell's encodings
initial_columns_count = df_encoded.shape[1]

# One-hot encode 'reported_satisfaction' if it exists
if 'reported_satisfaction' in df_encoded.columns:
    df_encoded = pd.get_dummies(df_encoded, columns=['reported_satisfaction'], prefix='satisfaction')
    print("Encoded 'reported_satisfaction'.")
else:
    print("Column 'reported_satisfaction' not found in df_encoded, skipping encoding.")

# One-hot encode 'operating_system' if it exists
if 'operating_system' in df_encoded.columns:
    df_encoded = pd.get_dummies(df_encoded, columns=['operating_system'], prefix='os')
    print("Encoded 'operating_system'.")
else:
    print("Column 'operating_system' not found in df_encoded, skipping encoding.")

final_columns_count = df_encoded.shape[1]
new_columns_added = final_columns_count - initial_columns_count

print(f"\nDataFrame shape after encoding: {df_encoded.shape}")
print(f"Number of new columns added in this step: {new_columns_added}")

KeyError: "None of [Index(['reported_satisfaction'], dtype='object')] are in the [columns]"

### ✍️ Your Response: 🔧
1.

# 🔧 Part 6: Reflection (100 words or less per question)

1. Which transformation do you think had the biggest impact on preparing your data for modeling?
2. Are there any features you created that you think will be especially useful for predicting churn?

### ✍️ Your Response: 🔧
1.
2.

## Export Your Notebook to Submit in Canvas
- Use the instructions from Lab 1

In [ ]:
!jupyter nbconvert --to html "lab_07_data_transformation.ipynb"